In [6]:
datasetX = torch.Tensor(10000, 2);
datasetY = torch.Tensor(10000, 1);

In [17]:
--[[counter = 1
start = 1
end = 400
val = -2
for (x in 1:500) {
  ass4df[start:end, 1] = rep(val, 400)
  ass4df[start:end, 2] = seq(-2,1.99, 0.01)
  #ass4df[counter, 3] = 2*x^2 + 3*y^2 + 1
  start = start + 400
  end = end + 400
  val = val + 0.01
} ]]
--start = 1;
--endd = 400;
vali = -2;
counter = 1
for i=1,100 do
    valj = -2;
    for j=1,100 do
        datasetX[counter][1] = vali;
        datasetX[counter][2] = valj;
        datasetY[counter] = 2*vali^2 - 3*valj^2 + 1;
        valj = valj + 0.1;
        counter = counter + 1;
    end
    vali = vali + 0.1;
end

In [32]:
datasetX[3031]

 1.0000
 1.0000
[torch.DoubleTensor of size 2]



In [49]:
y = torch.round(datasetY*10000)*0.00001; --round to 5 decimal places
print(y[3031])

 0
[torch.DoubleTensor of size 1]



In [53]:
f = torch.cat(y, datasetX, 2);
print(f[3031])

-0.0000
 1.0000
 1.0000
[torch.DoubleTensor of size 3]



In [ ]:
require 'torch'
require 'optim'
require 'nn'

-- We will write the loss to a text file and read from there to plot the loss as training proceeds
logger = optim.Logger('loss_log.txt')
--[[
data = torch.Tensor{
   {40,  6,  4},
   {44, 10,  4},
   {46, 12,  5},
   {48, 14,  7},
   {52, 16,  9},
   {58, 18, 12},
   {60, 22, 14},
   {68, 24, 20},
   {74, 26, 21},
   {80, 32, 24}
}]]
data = f;

model = nn.Sequential()                 -- define the container
ninputs = 2; noutputs = 6
model:add(nn.Linear(ninputs, noutputs)) -- define the only module
--model:add(nn.Tanh())
--model:add(nn.Linear(noutputs, 1))
criterion = nn.MSECriterion()
x, dl_dx = model:getParameters()

feval = function(x_new)
   -- set x to x_new, if differnt
   -- (in this simple example, x_new will typically always point to x,
   -- so the copy is really useless)
   if x ~= x_new then
      x:copy(x_new)
   end

   -- select a new training sample
   _nidx_ = (_nidx_ or 0) + 1
   if _nidx_ > (#data)[1] then _nidx_ = 1 end

   local sample = data[_nidx_]
   local target = sample[{ {1} }]      -- this funny looking syntax allows
   local inputs = sample[{ {2,3} }]    -- slicing of arrays.

   -- reset gradients (gradients are always accumulated, to accommodate 
   -- batch methods)
   dl_dx:zero()

   -- evaluate the loss function and its derivative wrt x, for that sample
   local loss_x = criterion:forward(model:forward(inputs), target)
   model:backward(inputs, criterion:backward(model.output, target))

   -- return loss(x) and dloss/dx
   return loss_x, dl_dx
end

sgd_params = {
   learningRate = 1e-3,
   learningRateDecay = 1e-4,
   weightDecay = 0,
   momentum = 0
}

for i = 1,1e4 do
   -- this variable is used to estimate the average loss
   current_loss = 0
   -- an epoch is a full loop over our training data
   for i = 1,(#data)[1] do
      _,fs = optim.sgd(feval,x,sgd_params)
      current_loss = current_loss + fs[1]
   end
   -- report average error on epoch
   current_loss = current_loss / (#data)[1]
   print('current loss = ' .. current_loss)
   
   logger:add{['training error'] = current_loss}
   logger:style{['training error'] = '-'}
   logger:plot()  
end



text = {40.32, 42.92, 45.33, 48.85, 52.37, 57, 61.82, 69.78, 72.19, 79.42}

print('id  approx   text')
for i = 1,(#data)[1] do
   local myPrediction = model:forward(data[i][{{2,3}}])
   print(string.format("%2d  %6.2f %6.2f", i, myPrediction[1], text[i]))
end
